## Training and Building the model

In [2]:
pip install tensorflow matplotlib numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


------------------------------------------------

In [7]:
import tensorflow as tf 
import numpy
from matplotlib import pyplot as plt

### loading dataset


In [9]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3

In [10]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
    
)

Found 16011 files belonging to 10 classes.
